# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_weather = pd.read_csv("../output_data/clean_cities.csv")
city_weather.head()
city_weather.dtypes

City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity      float64
Cloudiness    float64
Wind Speed    float64
Country        object
Date          float64
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps
gmaps.configure(api_key=g_key)
coordinates = city_weather[["Lat","Lng"]]
figure_layout = {
    'width': '800px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
heat_layer = gmaps.heatmap_layer(coordinates,weights = city_weather["Humidity"], dissipating=False,
                                 max_intensity=max(city_weather["Humidity"]),point_radius=2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
ideal_weather = city_weather.loc[(city_weather["Max Temp"]<75) & (city_weather["Max Temp"]>70),:]
ideal_weather = ideal_weather.loc[(ideal_weather["Cloudiness"]<10) & (ideal_weather["Wind Speed"]<8),:].reset_index(drop=True)
ideal_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,pedro juan caballero,-22.5472,-55.7333,73.36,23.0,0.0,4.61,PY,1.659215e+09
1,lagoa,39.0500,-27.9833,73.54,66.0,0.0,0.98,PT,1.659215e+09
2,fairbanks,64.8378,-147.7164,73.02,45.0,0.0,3.44,US,1.659215e+09
3,ponta do sol,32.6667,-17.1000,73.71,82.0,9.0,7.99,PT,1.659214e+09
4,saint-francois,46.4154,3.9054,71.04,53.0,0.0,2.30,FR,1.659215e+09
5,goderich,43.7501,-81.7165,72.91,53.0,6.0,3.44,CA,1.659215e+09
6,anqiu,36.4342,119.1925,73.56,92.0,4.0,2.17,CN,1.659215e+09
7,yuncheng,35.0231,110.9928,73.15,86.0,0.0,5.93,CN,1.659215e+09
8,caravelas,-17.7125,-39.2481,73.08,88.0,0.0,4.09,BR,1.659214e+09
9,auriflama,-20.6856,-50.5547,71.53,30.0,5.0,4.88,BR,1.659215e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = ideal_weather
hotel_df["Hotel"]=""

In [6]:
# base url
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# target_coordinates = lat,lng
target_radius = 5000
target_type = "lodging"

print("Beginning Data Retrieval")
print("-----------------------------")
for index,row in hotel_df.iterrows():
    city = row['City']
    lat = row['Lat']
    lng = row['Lng']
   # set up a parameters dictionary
    params = {
        "location": f'{lat},{lng}',
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }
    response = requests.get(url,params=params).json()
    try:
        print(f'Processing Record {index+1}| {city}')
        hotel = response['results'][0]['name']
        print(f'The closet hotel to {city} is {hotel}')
        hotel_df.loc[index,"Hotel"]=hotel
    except:
        print(f"{city} not found. Skipping...")
        pass

Beginning Data Retrieval
-----------------------------
Processing Record 1| pedro juan caballero
The closet hotel to pedro juan caballero is Hotel Pousada do Bosque
Processing Record 2| lagoa
The closet hotel to lagoa is Casa Das Faias
Processing Record 3| fairbanks
The closet hotel to fairbanks is Westmark Fairbanks Hotel & Conference Center
Processing Record 4| ponta do sol
The closet hotel to ponta do sol is Hotel do Campo
Processing Record 5| saint-francois
The closet hotel to saint-francois is Manoir Du Bonheur
Processing Record 6| goderich
The closet hotel to goderich is Harmony Inn
Processing Record 7| anqiu
The closet hotel to anqiu is Super 8 Anqiu Heping Road Coach Station
Processing Record 8| yuncheng
The closet hotel to yuncheng is 晋州快捷酒店
Processing Record 9| caravelas
The closet hotel to caravelas is Pousada dos Navegantes
Processing Record 10| auriflama
The closet hotel to auriflama is Capoia´s Hotel
Processing Record 11| waingapu
The closet hotel to waingapu is Pondok Wi

In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…